In [1]:
import torch
from omegaconf import OmegaConf
from src.models.blm.config import ModelArgs
from src.models.blm.pl_dataloader import TinyStoriesDataloader
from src.models.blm.pl_training import Transformer
from src.tokenize.tokenizer import Tokenizer

Seed set to 123
Seed set to 123


In [2]:
from src.models.blm.finetune.full_finetune import FineTuneBLM

In [3]:
from datasets import load_dataset, load_from_disk

ds = load_from_disk(
    "/home/pranav-pc/projects/OpenTransformer/multiformer/data/processed/TinyStories-Instruct-hf"
)

In [4]:
BASE_URL = "/home/pranav-pc/projects/OpenTransformer/multiformer"
tokenizer_path = BASE_URL + "/tokenizer_checkpoints"
tokenizer = Tokenizer(tokenizer_path)

In [5]:
MODEL_CONFIG_PATH = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/src/models/blm/conf/config.yaml"
)
MODEL_CHECKPOINT_PATH = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/blm-finetuned-tinnystories/last-v1.ckpt"
)

conf = OmegaConf.load(MODEL_CONFIG_PATH)

In [6]:
state_dict = torch.load(MODEL_CHECKPOINT_PATH)["state_dict"]
# Pytorch by default add _orig_mod in the checkpoint keys.#TODO: Take care of this while model checkpointing
state_dict = {k.replace("._orig_mod", ""): v for k, v in state_dict.items()}

In [7]:
config = ModelArgs(**conf["model"])
model = FineTuneBLM(Transformer(config))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
#### Inference
model.eval()
model = model.cuda()

In [69]:
ds["validation"][604]

{'text': 'User:\nWords: applaud, camp, lively\nSummary: A little girl named Lily enjoys playing and making friends at a lively camp in the forest.\nRandom sentence: She felt grateful for being at such a wonderful place.\nAssistant: Once upon a time, there was a big camp in the forest. Many children went there to play and have fun. The camp was very lively, with music and games all day long.\n\nOne day, a little girl named Lily did a cartwheel and everyone applauded her. She felt very happy and proud of herself. Lily loved being at the camp because she could make new friends and learn new things.\n\nAt night, they all sat around the campfire and sang songs. Lily loved the sound of the guitar and the happy faces of her friends. She felt grateful for being at such a wonderful place. Finally, it was time to go to sleep and Lily dreamed of all the fun things she would do the next day at the lively camp.'}

In [76]:
text = """User:\nWords: applaud, tent, lively\nSummary: A little girl named Lily enjoys playing and making friends at a lively camp in the forest.\nAssistant:"""

In [77]:
# text = "Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. It was sad because it couldn’t move. Every day, it would say"
# text = "Jack was hungry, so he went looking for"
# text = "Tim is a good boy. one day his father called and asked for the school exam result"
# text = "Jack wanted to read a book,so he went to"
# text = "My name is Mariama, my favorite"
# text = "User: Words: come, road, sad Summary: A bus becomes jealous of a shiny new car and undergoes an operation to go faster, becoming good friends with the car and making everyone in the town happy. Assistant:"
tokens = torch.LongTensor(tokenizer.encode(text)).to("cuda:0").view(1, -1)[:, :-1]
# print(tokens)
predicted_tokens = model.predict_step(
    tokens, None, max_new_tokens=1024, temperature=1.1, top_k=5, conditional_break=[2]
)[0].tolist()
# print(predicted_tokens)
print(tokenizer.decode_ids(predicted_tokens))

43
1
Once upon a time, there was a big tent in the middle of the forest. Lots of animals would gather around the tent and sleep. It was a lively camp with lots of singing birds and animals.

One day, a little girl named Lily went to the camp with her mom. They set up their camp and started to play. The animals would run and jump and make loud sounds. Lily loved the sound of them and clapped her hands.

After playing for a while, Lily and her mom sat at camp for a while. They watched the sun set and saw a big fire truck drive by. Lily was amazed and couldn't stop looking at it. She loved going to camp and having fun.


In [73]:
tokenizer.id_to_piece(13)

'<0x0A>'